In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

import sys, os, csv, datetime
from typing import Dict
parent_directory = os.path.split(os.getcwd())[0]
src_directory = os.path.join(parent_directory, 'src')
data_directory = os.path.join('/scratch/groups/gracegao', 'Google_GNSS_data')
ephemeris_data_directory = os.path.join(data_directory, 'ephemeris')
sys.path.insert(0, src_directory)

import matplotlib.pyplot as plt # plotting
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm.notebook import tqdm

import xarray as xr

/home/groups/gracegao/conda_envs/deep_gnss/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:27: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [3]:
from utils import *

from gnss_lib.sim_gnss import expected_measures, precompute_prange_satpos
import gnss_lib.sim_gnss as sgnss
from gnss_lib.utils import datetime_to_tow 
from gnss_lib import coordinates as coord

In [16]:
## HyperParams
data_config = {
    "old_root": os.path.join(data_directory, "simulated_matlab_clockcorr_half_small_rebatched/gnss_data"),
    "new_root": os.path.join(data_directory, "simulated_matlab_clockcorr_half_small_rebatched_bias_ctx")
    }

N_ctx = 3

ctx_mean = [0] + [100*np.random.rand() for _ in range(N_ctx)]
ctx_los = [np.array([0, 0, 1])]
for _ in range(N_ctx):
    vec = np.random.rand(3)
    ctx_los.append(vec/np.linalg.norm(vec))

bias_params = {
    "interval": 100,
    "sigma": 1
}

In [17]:
initialize_dirs(data_config["new_root"])
dir_paths = setup_dir(data_config["new_root"], ["gnss_data", "processed"])

In [18]:
dir_paths

['/scratch/groups/gracegao/Google_GNSS_data/simulated_matlab_clockcorr_half_small_rebatched_bias_ctx/gnss_data',
 '/scratch/groups/gracegao/Google_GNSS_data/simulated_matlab_clockcorr_half_small_rebatched_bias_ctx/processed']

In [19]:
debug_res_list = []
def debug_residuals(data, satXYZV, true_XYZb, prange_corr):
    delXYZ, true_range = sgnss._find_delxyz_range(satXYZV, true_XYZb[:3], len(satXYZV.index))
    return true_range + prange_corr + true_XYZb[3] - data['prange'].to_numpy()

list1 = sorted_list(data_config["old_root"])
traj_data = np.zeros((len(list1), 100, 4))

In [28]:
base_mean = ctx_mean[0]
base_los = ctx_los[0]
cur_interval = 0
for j in tqdm(range(len(list1)), desc='files'):
    filepath = list1[j]
    data = pd.read_hdf(filepath, 'new_data')
    outname = filepath.split('/')[-1]
    prange_corr = data['prange_corr'].to_numpy()
    bias_vec = np.zeros_like(prange_corr)
    
    losXYZ = np.vstack((data["sv_x"].to_numpy(), data["sv_y"].to_numpy(), data["sv_z"].to_numpy()))
    losXYZ = np.divide(losXYZ, np.linalg.norm(losXYZ, axis=0))
    
    if cur_interval > bias_params["interval"]:
        ctx_idx = np.random.randint(len(ctx_mean))
        base_mean = ctx_mean[ctx_idx]
        base_los = ctx_los[ctx_idx]
        cur_interval = 0
    cur_interval += 1
    
    ctx_dots = np.abs(base_los.reshape(1, -1) @ losXYZ)
    
    for i in range(len(bias_vec)):
        if bias_params is not None and ctx_dots[0, i]>0.6:
            bias_vec[i] = np.random.randn()*bias_params["sigma"] + base_mean

    print(bias_vec)
    data['prange'] = data['prange'].to_numpy() + bias_vec

    data.to_hdf(os.path.join(dir_paths[0], outname), key='new_data')

files:   0%|          | 3/1781 [00:00<00:59, 29.74it/s]

[ 1.26416732  0.          0.          0.          0.          0.
  0.         -0.3295499   0.          0.33328315]
[-0.26935307  0.          0.          0.          0.          0.
  0.          0.01181411  0.         -1.97327337]
[ 0.66838189  0.          0.          0.          0.          0.
  0.          0.91258214  0.         -2.5082171 ]
[ 0.88332362  0.          0.          0.          0.          0.
  0.          1.52880728  0.         -0.00563092]
[ 1.13914137  0.          0.          0.          0.          0.
  0.         -0.47895301  0.          0.87218696]
[-1.00489397  0.          0.          0.          0.          0.
  0.         -1.25445286  0.          0.00465333]
[ 0.92721547  0.          0.          0.          0.          0.
  0.         -0.29525525  0.         -1.23931706]


files:   1%|          | 11/1781 [00:00<00:51, 34.49it/s]

[ 2.20152443  0.          0.          0.          0.          0.
  0.         -0.17483199  0.          0.70868678]
[-1.18609587  0.          0.          0.          0.          0.
  0.          1.22822861  0.         -0.72157073]
[-0.45142021  0.          0.          0.          0.          0.
  0.          0.42315184  0.          0.80790333]
[-0.04941963  0.          0.          0.          0.          0.
  0.         -1.29787095  0.          0.03435854]
[-0.20850064  0.          0.          0.          0.          0.
  0.          0.5908932   0.          0.52292467]
[0.04767269 0.         0.         0.         0.         0.
 0.         0.31091111 0.         0.3463632 ]
[-0.09604527  0.          0.          0.          0.          0.
  0.         -0.07684541  0.         -0.90366055]
[ 0.81907026  0.          0.          0.          0.          0.
  0.         -0.94829933  0.         -0.5786315 ]


files:   1%|          | 19/1781 [00:00<01:02, 28.22it/s]

...

files: 100%|█████████▉| 1777/1781 [03:02<00:00,  9.33it/s]

[79.82854622  0.         80.80611955  0.          0.         81.4723548
  0.         77.98332985  0.         80.25071317]
[80.21660117  0.         80.95540012  0.          0.         81.78957733
  0.         79.29782827  0.         80.34058894]
[79.22215471  0.         79.25907326  0.          0.         80.57455784
  0.         79.71805624  0.         80.86770497]


files: 100%|█████████▉| 1780/1781 [03:03<00:00,  8.84it/s]

[79.77239574  0.         82.25197058  0.          0.         81.46199366
  0.         81.83848369  0.         80.88501665]
[78.95774161  0.         81.48313257  0.          0.         79.96126801
  0.         81.02480949  0.         80.5225582 ]


files: 100%|██████████| 1781/1781 [03:03<00:00,  9.71it/s]


In [13]:
bias_vec

array([  0.        ,   0.        ,   0.        , 850.25833663,
         0.        ,   0.        ,   0.        , 850.26884429,
         0.        ,   0.        ])

In [ ]:
for j in tqdm(range(len(list1)), desc='files'):
    filepath = list1[j]
    fulldata = pd.read_csv(filepath)
    outname = filepath.split('/')[-1][:-4]
    list2 = fulldata['t_idx'].unique()
    for i in range(len(list2)):
        timestep = list2[i]
        data = pd.read_hdf(os.path.join(dir_paths[0], outname+'_'+str(i)+'.h5'), 'new_data')

        # Calculate corrections
        gpsweek, tow = datetime_to_tow(pd.to_datetime(timestep))
        ephem = data.set_index('sv')
        _data0 = data.iloc[0]

        # Change to WLS or some other approximate initialization
        true_XYZb = np.array([_data0['Rxx'], _data0['Rxy'], _data0['Rxz'], _data0['b'], _data0['Rxvx'], _data0['Rxvy'], _data0['Rxvz'], _data0['b_dot']])

        # Generate expected measures and satellite positions/velocities
        prange_corr, satXYZV = precompute_prange_satpos(gpsweek, tow, ephem, true_XYZb[:3], true_XYZb[3], true_XYZb[7], true_XYZb[4:7])

        bias_vec = np.zeros_like(prange_corr)
        
        for i in range(len(bias_vec)):
            if bias_params is not None and np.random.rand() < bias_params["prob"]:
                bias = np.random.randn()*bias_params["sigma"] + bias_params["mean"]
            else:
                bias = 0.0
            bias_vec[i] = bias 
        
        tmp_res = debug_residuals(data, satXYZV, true_XYZb, prange_corr)
        debug_res_list.append(tmp_res)
        
        traj_data[j, i, :] = true_XYZb[:4]
        sim_data = {
            "timestamp": data['t_idx'].to_numpy(), 
            "gt_x": data['Rxx'].to_numpy(),
            "gt_y": data['Rxy'].to_numpy(),
            "gt_z": data['Rxz'].to_numpy(),
            "gt_b": data['b'].to_numpy(),
            "gt_vx": data['Rxvx'].to_numpy(),
            "gt_vy": data['Rxvy'].to_numpy(),
            "gt_vz": data['Rxvz'].to_numpy(),
            "gt_vb": data['b_dot'].to_numpy(),
            "prange": data['prange'].to_numpy() + bias_vec, 
            "prange_corr": prange_corr, 
            "doppler": data['doppler'].to_numpy(), 
            "sv_x": satXYZV['x'].to_numpy(),
            "sv_y": satXYZV['y'].to_numpy(),
            "sv_z": satXYZV['z'].to_numpy(),
            "sv_vx": satXYZV['vx'].to_numpy(),
            "sv_vy": satXYZV['vy'].to_numpy(),
            "sv_vz": satXYZV['vz'].to_numpy(),
                   }
    
        new_data = pd.DataFrame(data=sim_data)
        
        new_data.to_hdf(os.path.join(dir_paths[0], outname+'_'+str(i)+'.h5'), key='new_data')

In [ ]:
# # PLOT RESIDUALS
# for i, res_l in enumerate(debug_res_list):
#     plt.scatter([i]*len(res_l), res_l)

# # PLOT TRAJECTORY
# i = 1

# plt.figure()
# plt.plot(traj_data[i, :, 0])

# plt.figure()
# plt.plot(traj_data[i, :, 1])

# plt.figure()
# plt.plot(traj_data[i, :, 2])

# plt.figure()
# plt.plot(traj_data[i, :, 3])

In [ ]:
tmp = pd.read_hdf('/scratch/groups/gracegao/Google_GNSS_data/simulated_matlab_clockcorr_half_small_rebatched_bias_ctx/gnss_data/0_002_1_60.h5', 'new_data')

In [ ]:
tmp